In [ ]:
# package import

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
# connect google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Insert Data
!unzip /content/drive/MyDrive/FYP/Data_sample/Sample_Fish_Dataset.zip -d /content/

Archive:  /content/drive/MyDrive/FYP/Data_sample/Sample_Fish_Dataset.zip
   creating: /content/Sample_Fish_Dataset/
   creating: /content/Sample_Fish_Dataset/train/
   creating: /content/Sample_Fish_Dataset/train/fin_rot/
  inflating: /content/Sample_Fish_Dataset/train/fin_rot/train_finrot (1).jfif  
  inflating: /content/Sample_Fish_Dataset/train/fin_rot/train_finrot (10).jfif  
  inflating: /content/Sample_Fish_Dataset/train/fin_rot/train_finrot (11).jfif  
  inflating: /content/Sample_Fish_Dataset/train/fin_rot/train_finrot (12).jfif  
  inflating: /content/Sample_Fish_Dataset/train/fin_rot/train_finrot (13).png  
  inflating: /content/Sample_Fish_Dataset/train/fin_rot/train_finrot (14).jfif  
  inflating: /content/Sample_Fish_Dataset/train/fin_rot/train_finrot (15).jfif  
  inflating: /content/Sample_Fish_Dataset/train/fin_rot/train_finrot (16).jfif  
  inflating: /content/Sample_Fish_Dataset/train/fin_rot/train_finrot (17).png  
  inflating: /content/Sample_Fish_Dataset/train/fin_

In [ ]:
# Paths & Parameters
train_dir = '/content/Sample_Fish_Dataset/train'
validation_dir = '/content/Sample_Fish_Dataset/validation'
IMG_SIZE = (224, 224) # MobileNetV2  standard size
BATCH_SIZE = 32

# create ImageDataGenerator  (rescale only)
datagen = ImageDataGenerator(rescale=1./255)

# Training data generator
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False # Feature extraction must be False
)

# Validation data generator
validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 367 images belonging to 4 classes.
Found 81 images belonging to 4 classes.


In [ ]:
# MobileNetV2 base model  loading
base_model = MobileNetV2(input_shape=(224, 224, 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False

# create final  feature extractor
feature_extractor = Model(inputs=base_model.input,
                          outputs=GlobalAveragePooling2D()(base_model.output))

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
print("Extracting features from training data...")
train_features = feature_extractor.predict(train_generator)

print("Extracting features from validation data...")
val_features = feature_extractor.predict(validation_generator)

# get  Labels in generator
train_labels = train_generator.classes
val_labels = validation_generator.classes

print("Feature extraction complete!")

Extracting features from training data...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12/12 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step
Extracting features from validation data...
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step
Feature extraction complete!


In [ ]:

print("Shape of extracted training features:", train_features.shape)
print("Shape of training labels:", train_labels.shape)
print("-" * 30)
print("Shape of extracted validation features:", val_features.shape)
print("Shape of validation labels:", val_labels.shape)

Shape of extracted training features: (367, 1280)
Shape of training labels: (367,)
------------------------------
Shape of extracted validation features: (81, 1280)
Shape of validation labels: (81,)


Train SVM Model
# New Section

In [ ]:


from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC

# Define the parameters we want to test as a dictionary
param_grid = [
    {'C': [0.1, 1, 10, 100], 'kernel': ['linear']},
    {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.001], 'kernel': ['rbf']}
]

# Create the GridSearchCV object (verbose=2 shows the process)
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)

# Train the GridSearchCV object
print("Starting GridSearchCV... This might take a few minutes.")
grid.fit(train_features, train_labels)

# Check what the best parameters are
print("\nBest parameters found: ", grid.best_params_)

# Get predictions from the best model
grid_predictions = grid.predict(val_features)

# Check the accuracy and report of the best model
print(f"\nBest Model Validation Accuracy: {accuracy_score(val_labels, grid_predictions) * 100:.2f}%")

# For now, we'll view the report with numbers. If needed, the class names can be provided manually.
class_names = ['fin_rot', 'hole_in_head', 'pop_eye', 'white_spot']
print("\nClassification Report for the Best Model:")
print(classification_report(val_labels, grid_predictions, target_names=class_names))

Starting GridSearchCV... This might take a few minutes.
Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END ...............................C=0.1, kernel=linear; total time=   0.1s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.1s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.1s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.1s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.1s
[CV] END .................................C=1, kernel=linear; total time=   0.1s
[CV] END .................................C=1, kernel=linear; total time=   0.1s
[CV] END .................................C=1, kernel=linear; total time=   0.1s
[CV] END .................................C=1, kernel=linear; total time=   0.1s
[CV] END .................................C=1, kernel=linear; total time=   0.1s
[CV] END ................................C=10, kernel=linear; total time=